In [ ]:
# ============================================
# 올리브영 Top100 전체 제품 OCR (txt + xlsx)
# ============================================
import os
import glob
from paddleocr import PaddleOCR
import pandas as pd
import re
from datetime import datetime

# ============================================
# 1. 경로 설정
# ============================================
IMG_BASE_DIR = r"C:\py_temp\올리브영_이미지2025-11-25-18-11-12-올리브영-이미지"
OUTPUT_DIR = r"C:\Users\itwill\Desktop\올리브영\중간프로젝트 코스메틱"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ============================================
# 2. PaddleOCR 모델 로딩
# ============================================
print("🔧 PaddleOCR 모델 로딩 중...")
ocr = PaddleOCR(
    use_angle_cls=True,
    lang='korean'
)
print("✅ 모델 로딩 완료!\n")

# ============================================
# 3. 강화된 텍스트 정제 함수
# ============================================
def clean_text(text):
    """OCR 결과 정제"""
    text = re.sub(r'\d+\.?\d*\s?(ml|oz|g|mg|kg|l|fl|f1)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d+\.?\d*\b', '', text)
    text = re.sub(r'\d+\.?\d*/\d+\.?\d*', '', text)
    text = re.sub(r'[^\w\s가-힣a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    words = text.split()
    filtered_words = []
    
    for word in words:
        if len(word) == 1 and re.match(r'[가-힣ㄱ-ㅎㅏ-ㅣ]', word):
            continue
        if len(word) == 1 and re.match(r'[a-zA-Z]', word):
            continue
        if word.isdigit():
            continue
        if len(word) >= 2:
            filtered_words.append(word)
    
    return ' '.join(filtered_words).strip()

# ============================================
# 4. 모든 제품 폴더 찾기
# ============================================
product_folders = sorted([
    d for d in os.listdir(IMG_BASE_DIR) 
    if os.path.isdir(os.path.join(IMG_BASE_DIR, d))
])

print(f"📁 발견된 제품 폴더: {len(product_folders)}개")
print(f"   예시: {product_folders[:5]}\n")

if len(product_folders) == 0:
    print("⚠️ 제품 폴더가 없습니다!")
    exit()

# ============================================
# 5. 전체 통합 결과 저장용
# ============================================
all_products_summary = []
total_images_processed = 0
total_words_extracted = 0

# ============================================
# 6. 각 폴더별로 OCR 수행
# ============================================
print("=" * 80)
print("🚀 전체 제품 OCR 시작!")
print("=" * 80)

for folder_idx, folder_name in enumerate(product_folders, start=1):
    print(f"\n{'='*80}")
    print(f"📦 [{folder_idx}/{len(product_folders)}] {folder_name} 폴더 처리 중...")
    print(f"{'='*80}")
    
    IMG_DIR = os.path.join(IMG_BASE_DIR, folder_name)
    
    img_exts = ['*.png', '*.jpg', '*.jpeg', '*.webp']
    img_files = []
    
    for ext in img_exts:
        img_files.extend(glob.glob(os.path.join(IMG_DIR, ext)))
    
    img_files = sorted(img_files)
    
    print(f"   📷 이미지 개수: {len(img_files)}장")
    
    if len(img_files) == 0:
        print(f"   ⚠️ 이미지 없음, 다음 폴더로 이동\n")
        continue
    
    image_results = []
    product_text_lines = []
    
    for img_idx, img_path in enumerate(img_files, start=1):
        img_name = os.path.basename(img_path)
        
        try:
            result = ocr.ocr(img_path)
            
            raw_lines = []
            clean_lines = []
            
            if result and len(result) > 0:
                ocr_result = result[0]
                
                if hasattr(ocr_result, 'rec_texts') and hasattr(ocr_result, 'rec_scores'):
                    texts = ocr_result.rec_texts
                    scores = ocr_result.rec_scores
                    
                    for text, score in zip(texts, scores):
                        if score >= 0.4:
                            raw_lines.append(f"{text} (신뢰도: {score:.2f})")
                            cleaned = clean_text(text)
                            if cleaned:
                                clean_lines.append(cleaned)
                
                elif isinstance(ocr_result, dict):
                    if 'rec_texts' in ocr_result and 'rec_scores' in ocr_result:
                        texts = ocr_result['rec_texts']
                        scores = ocr_result['rec_scores']
                        
                        for text, score in zip(texts, scores):
                            if score >= 0.4:
                                raw_lines.append(f"{text} (신뢰도: {score:.2f})")
                                cleaned = clean_text(text)
                                if cleaned:
                                    clean_lines.append(cleaned)
            
            image_results.append({
                "이미지": img_name,
                "원본텍스트": " ".join([line.split(" (신뢰도:")[0] for line in raw_lines]),
                "정제텍스트": " ".join(clean_lines),
                "줄수": len(clean_lines)
            })
            
            product_text_lines.extend(clean_lines)
            total_images_processed += 1
            
            if img_idx % 5 == 0 or img_idx == len(img_files):
                print(f"      진행: {img_idx}/{len(img_files)} 이미지 완료", end='\r')
        
        except Exception as e:
            print(f"      ⚠️ {img_name} 처리 실패: {e}")
            image_results.append({
                "이미지": img_name,
                "원본텍스트": f"[오류: {e}]",
                "정제텍스트": "",
                "줄수": 0
            })
            continue
    
    df_product = pd.DataFrame(image_results)
    excel_path = os.path.join(OUTPUT_DIR, f"{folder_name}_ocr.xlsx")
    df_product.to_excel(excel_path, index=False)
    
    product_combined_text = " ".join(product_text_lines)
    txt_path = os.path.join(OUTPUT_DIR, f"{folder_name}_ocr.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(product_combined_text)
    
    all_products_summary.append({
        "제품번호": folder_name,
        "이미지수": len(img_files),
        "추출단어수": len(product_text_lines),
        "텍스트": product_combined_text
    })
    
    total_words_extracted += len(product_text_lines)
    
    print(f"\n   ✅ {folder_name} 완료: {len(img_files)}장 → {len(product_text_lines)}단어 추출")
    print(f"   💾 저장: {folder_name}_ocr.txt, {folder_name}_ocr.xlsx")
    
    if folder_idx % 10 == 0:
        temp_df = pd.DataFrame(all_products_summary)
        temp_df.to_excel(os.path.join(OUTPUT_DIR, "통합결과_중간저장.xlsx"), index=False)
        print(f"\n   💾 중간 저장 완료 ({folder_idx}개 처리됨)\n")

print("\n" + "=" * 80)
print("📊 전체 통합 파일 생성 중...")
print("=" * 80)

df_summary = pd.DataFrame(all_products_summary)
summary_excel_path = os.path.join(OUTPUT_DIR, "전체제품_통합요약.xlsx")
df_summary.to_excel(summary_excel_path, index=False)
print(f"✅ 통합 요약 엑셀: {summary_excel_path}")

all_text = " ".join([item["텍스트"] for item in all_products_summary])
all_txt_path = os.path.join(OUTPUT_DIR, "전체제품_통합텍스트.txt")
with open(all_txt_path, "w", encoding="utf-8") as f:
    f.write(all_text)
print(f"✅ 통합 txt: {all_txt_path}")

print("\n" + "=" * 80)
print("🎉 전체 OCR 작업 완료!")
print("=" * 80)
print(f"📊 처리 결과:")
print(f"   - 처리된 제품: {len(product_folders)}개")
print(f"   - 처리된 이미지: {total_images_processed}장")
print(f"   - 추출된 단어: {total_words_extracted:,}개")
print(f"   - 총 텍스트량: {len(all_text):,}자")
print(f"\n📁 결과 위치: {OUTPUT_DIR}")
print("=" * 80)

print("\n📈 제품별 통계 (상위 10개):")
df_sorted = df_summary.sort_values('추출단어수', ascending=False)
print(df_sorted[['제품번호', '이미지수', '추출단어수']].head(10).to_string(index=False))